#import guardrails

In [13]:
import os
from hr__guardrails import (
    violates_safety_policy,
    is_hr_question,
    is_answer_grounded,
    contains_sensitive_advice,
    is_query_too_long
)


Load PDFs (with OCR fallback)

In [14]:
from langchain_community.document_loaders import PyPDFLoader, UnstructuredPDFLoader

def load_all_pdfs(pdf_folder_path):
    documents = []

    for file_name in os.listdir(pdf_folder_path):
        if not file_name.endswith(".pdf"):
            continue

        full_path = os.path.join(pdf_folder_path, file_name)

        loader = PyPDFLoader(full_path)
        pages = loader.load()

        extracted_text = "".join([p.page_content.strip() for p in pages])

        if not extracted_text:
            print(f"OCR applied for: {file_name}")
            loader = UnstructuredPDFLoader(full_path, strategy="ocr_only")
            pages = loader.load()

        for doc in pages:
            doc.metadata["source"] = file_name

        documents.extend(pages)

    return documents


pdf_folder_path = r"C:\Users\91880\Desktop\GenAI_Projects\PuchoHR\Data\documents"
docs = load_all_pdfs(pdf_folder_path)
print("Total pages:", len(docs))


Total pages: 70


Chunking

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=256
)

chunks = text_splitter.split_documents(docs)
print("Total chunks:", len(chunks))


Total chunks: 512


Create & Persist Vector DB using ollama

In [16]:
'''from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

embedding = OllamaEmbeddings(model="nomic-embed-text")

vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    persist_directory="./chroma_db"
)

vector_db.persist()
print("Vector DB created and persisted.")'''


'from langchain_ollama import OllamaEmbeddings\nfrom langchain_community.vectorstores import Chroma\n\nembedding = OllamaEmbeddings(model="nomic-embed-text")\n\nvector_db = Chroma.from_documents(\n    documents=chunks,\n    embedding=embedding,\n    persist_directory="./chroma_db"\n)\n\nvector_db.persist()\nprint("Vector DB created and persisted.")'

using huggingface model

In [17]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-MiniLM-L6-v2"
)

vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    persist_directory="./chroma_db"
)

vector_db.persist()
print("Vector DB created and persisted with Hugging Face embeddings.")


Vector DB created and persisted with Hugging Face embeddings.


Load Vector DB + Models

In [18]:
'''
from sentence_transformers import SentenceTransformer, util
from langchain_ollama import OllamaLLM

embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

vector_db = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embedding
)

llm = OllamaLLM(
    model="llama3.2",
    temperature=0.2
)
'''


'\nfrom sentence_transformers import SentenceTransformer, util\nfrom langchain_ollama import OllamaLLM\n\nembedding_model = SentenceTransformer(\'paraphrase-MiniLM-L6-v2\')\n\nvector_db = Chroma(\n    persist_directory="./chroma_db",\n    embedding_function=embedding\n)\n\nllm = OllamaLLM(\n    model="llama3.2",\n    temperature=0.2\n)\n'

In [19]:
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

pipe = pipeline(
    "text2text-generation",   # IMPORTANT for T5/FLAN models
    model=model,
    tokenizer=tokenizer,
    max_length=512
)

llm = HuggingFacePipeline(pipeline=pipe)


Device set to use cpu


HR Bot Function

In [20]:
def ask_hr(question):
    if is_query_too_long(question):
        return "Your query is too long, please shorten it."

    if violates_safety_policy(question):
        return "Sorry, I cannot answer that question due to safety policy."

    if not is_hr_question(question):
        return "Sorry, this question is not related to HR policies."

    docs = vector_db.similarity_search(question, k=4)
    if not docs:
        return "Information not found in documents."

    context = "\n\n".join(
        f"Source: {doc.metadata['source']}\n{doc.page_content}"
        for doc in docs
    )

    prompt = f"""
You are an HR assistant.
Answer ONLY using the provided context.
If the answer is not present, say:
"Information not found in documents."

Context:
{context}

Question:
{question}

Answer:
"""

    answer = llm.invoke(prompt)

    if not is_answer_grounded(answer, docs):
        return "Information not found in documents."

    answer_emb = embedding_model.encode(str(answer))
    context_text = " ".join(doc.page_content for doc in docs)
    context_emb = embedding_model.encode(context_text)
    similarity = util.cos_sim(answer_emb, context_emb).item()

    if similarity < 0.8:
        return "Answer may not be fully accurate based on available documents."

    if contains_sensitive_advice(answer):
        return "I cannot provide legal or medical advice. Please consult the appropriate professional."

    return answer


Database + Test

In [21]:
from db_util import init_db, store_chat, get_recent_chat

init_db()

session_id = "user123"
question = "What is the leave policy for regular employees?"

answer = ask_hr(question)
print("LLM answer:", answer)

store_chat(session_id, question, answer, source_doc="HR_Policy-GGIAL.pdf", similarity_score=0.92)

previous_chats = get_recent_chat(session_id)
print(previous_chats)


Token indices sequence length is longer than the specified maximum sequence length for this model (574 > 512). Running this sequence through the model will result in indexing errors


LLM answer: Casual & Sick leave shall be credited on 1st January of every Year. • Leave must be applied through electronic leave management in HR4U Portal and sanction to be taken before proceeding on leave. • Leave for Pricipal Associates and Advisors will be as per the contract terms. Type Eligibility Procedure Casual Leave Employees on regular rolls, Consultants & Advisors - 12 days per year; will lapse at the end of the year - CL can be availed for half day also Source: HR_Policy-GGIAL.pdf 1st January, and 1st July, every year. Casual & Sick leave shall be credited on 1st January of every Year. • Leave must be applied through electronic leave management in HR4U Portal and sanction to be taken before proceeding on leave. • Leave for Pricipal Associates and Advisors will be as per the contract terms. Type Eligibility Procedure Casual Leave Employees on regular rolls, Consultants & Advisors - 12 days per year; will lapse at the end of the year - CL can be availed for half day also Sou

In [22]:
# Ask multiple questions
q1 = "What is the leave policy for regular employees?"
a1 = ask_hr(q1)
store_chat("user123", q1, a1, source_doc="HR_Policy-GGIAL.pdf", similarity_score=0.90)

q2 = "What is the probation period?"
a2 = ask_hr(q2)
store_chat("user123", q2, a2, source_doc="HR_Policy-GGIAL.pdf", similarity_score=0.88)

print("Q1:", a1)
print("Q2:", a2)

# Fetch last chats
history = get_recent_chat("user123", limit=5)
print("\nRecent Chat History:")
for q, a in history:
    print("Q:", q)
    print("A:", a)
    print("-" * 50)


Q1: Casual & Sick leave shall be credited on 1st January of every Year. • Leave must be applied through electronic leave management in HR4U Portal and sanction to be taken before proceeding on leave. • Leave for Pricipal Associates and Advisors will be as per the contract terms. Type Eligibility Procedure Casual Leave Employees on regular rolls, Consultants & Advisors - 12 days per year; will lapse at the end of the year - CL can be availed for half day also Source: HR_Policy-GGIAL.pdf 1st January, and 1st July, every year. Casual & Sick leave shall be credited on 1st January of every Year. • Leave must be applied through electronic leave management in HR4U Portal and sanction to be taken before proceeding on leave. • Leave for Pricipal Associates and Advisors will be as per the contract terms. Type Eligibility Procedure Casual Leave Employees on regular rolls, Consultants & Advisors - 12 days per year; will lapse at the end of the year - CL can be availed for half day also Source: HR_